In [1]:
#!pip install keybert

In [2]:
#!pip install bert-extractive-summarizer

In [1]:
import re
import os, os.path
import fnmatch
import json
import collections
import glob
import json


In [2]:
book ="SOMEWT"
def get_year_to_content(book):
    dir_path = './%s/part'% book
    date_dict = {}
    year_mode_dict = {}
    # for part in range(1,Parts+1):
    rootdir = r"C:\Users\hp\OneDrive\Documents\topic_coherence\%s"% book
    part = 1
    for path in glob.glob(f'{rootdir}/*/**/', recursive=True):
        x = 1
        #print("part:" + str(part))
        date_dict[str(part)] = {}
        year_mode_dict[str(part)] = {}
        #print(dir_path + str(part) + '/')
        no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
    #     print('no.of_chapters:' + str(no_of_chapters))
        while x <= no_of_chapters:
            target_x = no_of_chapters+1
            for i in range(x,target_x):
                #print("chapter:" + str(i))
                with open(dir_path + str(part) + '/chapter'+ str(i) + '.txt', 'r', encoding='utf-8') as content_file:
                    content = content_file.read()

                regex= "\d{4}"
                dates = re.findall(regex, content)
                mdates = []
                year_list = []
                try:
                    for date in dates:
                        if int(date) < 1960 and int(date) > 1800:                    
                            #print date
                            year_list.append(date)
                            mdates.append(date)                       
                except:
                    pass
                date_dict[str(part)][str(i)] = mdates
                try:
                    year_mode_dict[str(part)][str(i)] = max(set(year_list), key=year_list.count)
                except:
                    year_mode_dict[str(part)][str(i)] = -1
            x = target_x
            part += 1

    # assigning chapter the date with closest chapter
    def closest(lst, K):
        return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]


    #for part in range(1,Parts+1):
    part = 1
    for path in glob.glob(f'{rootdir}/*/**/', recursive=True):
        x = 1
        # checking which chapters has date assigned
        no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
        chapters_with_date = []
        while x <= no_of_chapters:
            target_x = no_of_chapters+1
            for i in range(x,target_x):
                # print("chapter:" + str(i))
                if year_mode_dict[str(part)][str(i)] != -1:
                    chapters_with_date.append(i)
            x = target_x
        x = 1
        # assigning date to chapter which is closest to it
        while x <= no_of_chapters:
            target_x = no_of_chapters+1
            for i in range(x,target_x):
                #print("chapter:" + str(i))
                k = closest(chapters_with_date,i)
                #print("K:",k)
                year_mode_dict[str(part)][str(i)] = year_mode_dict[str(part)][str(k)]
            x = target_x
        part += 1

    rootdir = r"C:\Users\hp\OneDrive\Documents\topic_coherence\SOMEWT"
    year_to_chap = collections.defaultdict(list)  # year -> list of folder paths
    for key in year_mode_dict.keys():
        folder = f"{rootdir}\part{key}"
        for k2 in year_mode_dict[key].keys():
            file = folder
            file += f"\chapter{k2}.txt"
            val = year_mode_dict[key][k2]
            year_to_chap[val].append(file)

    #print(year_to_chap)
    year_to_content = {}
    for key in year_to_chap.keys():
    #     print(key)
        content = ""
        for l in year_to_chap[key]:
    #         print(l)
            with open(l,'r') as f:
                content += f.read()
            year_to_content[key] = content

    return year_to_content

In [3]:
year_to_content = get_year_to_content(book)
sort_keys = [int(k) for k in year_to_content.keys()]
sort_keys.sort()
sort_keys = [str(k) for k in year_to_content.keys()]

year_to_content2 = {}
for i in sorted(sort_keys):
    year_to_content2[i]= year_to_content[i]
with open("year_to_content.json", "w") as outfile:
    json.dump(year_to_content2, outfile)

# Keyword Extraction
### Finally, we get the year to content mapping
Now we will see keyword extraction first

In [6]:
def extract_keyword_from_date(doc):
    from keybert import KeyBERT
    kw_model = KeyBERT()
    onegram_keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1, 1),top_n=3,stop_words='english',use_mmr = True, diversity=0.5)
    twogram_keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1, 2),top_n=3,stop_words='english',use_mmr = True, diversity=0.5)
    threegram_keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(3, 3),top_n=3,stop_words='english',use_mmr = True, diversity=0.5)
    onegram_keywords2 = [i[0] for i in onegram_keywords]
    twogram_keywords2 = [i[0] for i in twogram_keywords]
    threegram_keywords2 = [i[0] for i in threegram_keywords]
    return onegram_keywords2,twogram_keywords2,threegram_keywords2

## Text Summarization with Bert

In [5]:
from summarizer import Summarizer,TransformerSummarizer
year_to_summarizer = {}

bert_model = Summarizer()

for key in year_to_content.keys():
    year_to_summarizer[key] = ''.join(bert_model(year_to_content[key], min_length=40))


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Text Summarization with GPT-2

In [6]:
# GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
# full = ''.join(GPT2_model(doc, min_length=40))
# print(full)

In [7]:
sort_keys = [int(k) for k in year_to_summarizer.keys()]
sort_keys.sort()
sort_keys = [str(k) for k in year_to_summarizer.keys()]

year_to_summarizer2 = {}
for i in sorted(sort_keys):
    year_to_summarizer2[i]= year_to_summarizer[i]

In [8]:
with open("year_to_summarizer.json", "w") as outfile:
    json.dump(year_to_summarizer2, outfile)

In [9]:
with open('year_to_summarizer.json') as json_file:
    year_to_summarizer = json.load(json_file)

In [10]:
year_to_summarizer.keys()

dict_keys(['1857', '1869', '1887', '1888', '1890', '1891', '1893', '1894', '1896', '1897', '1898', '1900', '1901', '1904', '1906', '1908', '1911', '1912', '1914', '1915', '1917', '1919', '1920', '1921', '1924'])

In [11]:
year_to_content.keys()

dict_keys(['1869', '1887', '1890', '1891', '1857', '1893', '1924', '1888', '1894', '1896', '1897', '1906', '1901', '1898', '1914', '1904', '1900', '1912', '1911', '1915', '1917', '1919', '1908', '1920', '1921'])

In [12]:
# timeline_content_map = get_timeline_interval_content(5)

## year_to_keywords json

In [2]:
with open("year_to_content.json") as outfile:
    year_to_content = json.load(outfile)

In [3]:
with open('year_to_summarizer.json') as json_file:
    year_to_summarizer = json.load(json_file)

In [4]:
year = {
        "chapterwise": {
            "1gram":[],
            "2gram":[],
            "3gram":[]
        },
        "summarywise":{
            "1gram":[],
            "2gram":[],
            "3gram":[]     
        }
}

# data = {}
# # initialization
# for key in year_to_content.keys():
#     data[key] = year
    

In [7]:
import copy
data = {}
for key in year_to_content.keys():
    onegram_keywords,twogram_keywords,threegram_keywords = extract_keyword_from_date(year_to_summarizer[key])
    onegram_keywords2,twogram_keywords2,threegram_keywords2 = extract_keyword_from_date(year_to_content[key])
    data = copy.deepcopy(data)
    data[key] = year
    data[key]['chapterwise']['1gram'] = onegram_keywords2
    data[key]["chapterwise"]['2gram'] = twogram_keywords2
    data[key]["chapterwise"]['3gram'] = threegram_keywords2
    data[key]["summarywise"]['1gram'] = onegram_keywords
    data[key]["summarywise"]['2gram'] = twogram_keywords
    data[key]["summarywise"]['3gram'] = threegram_keywords
    print(f"{key}  -> {onegram_keywords}   -> {data[key]['chapterwise']['1gram']}")

1857  -> ['dadabhai', 'reading', 'venerable']   -> ['law', 'hindu', 'practice']
1869  -> ['gandhi', 'gujarati', 'pity']   -> ['gandhi', 'karamchand', 'widowed']
1887  -> ['meat', 'acquaintance', 'weaknesses']   -> ['friendships', 'despaired', 'eldest']
1888  -> ['transvaal', 'exempted', 'walk']   -> ['transvaal', 'exemption', 'satyagraha']
1890  -> ['vegetarianism', 'definitions', 'vow']   -> ['vegetarianism', 'internally', 'intervals']
1891  -> ['communion', 'barristers', 'deferred']   -> ['dinners', 'deferred', 'rupee']
1893  -> ['grief', 'raychandbhai', 'affidavit']   -> ['sea', 'mehta', 'agitation']
1894  -> ['pay', 'mahatmaship', 'collections']   -> ['congress', 'natal', 'pandit']
1896  -> ['servant', 'urdu', '1896']   -> ['servant', 'household', 'madras']
1897  -> ['parsi', 'durban', 'manners']   -> ['dress', 'hindu', 'banias']
1898  -> ['diet', 'restraint', 'proverb']   -> ['writings', 'dietetics', 'gujarati']
1900  -> ['brahmacharya', 'thoughts', 'zulu']   -> ['zulus', 'rebelli

In [8]:
data

{'1857': {'chapterwise': {'1gram': ['law', 'hindu', 'practice'],
   '2gram': ['practise law', 'studied shakespeare', 'know indian'],
   '3gram': ['learnt indian law',
    'making successful lawyer',
    'legal maxims did']},
  'summarywise': {'1gram': ['dadabhai', 'reading', 'venerable'],
   '2gram': ['indian history', 'introduction dadabhai', 'read kaye'],
   '3gram': ['introduction dadabhai thought',
    'know indian history',
    'hold read books']}},
 '1869': {'chapterwise': {'1gram': ['gandhi', 'karamchand', 'widowed'],
   '2gram': ['gandhi father', 'leave porbandar', 'children marry'],
   '3gram': ['kaba gandhi father',
    'compelled leave porbandar',
    'kathiawad states uttamchand']},
  'summarywise': {'1gram': ['gandhi', 'gujarati', 'pity'],
   '2gram': ['gandhi grandfather', 'bania caste', 'vow illness'],
   '3gram': ['gandhis belong bania',
    'wife pensioner rajkot',
    'chandrayana2 vow illness']}},
 '1887': {'chapterwise': {'1gram': ['friendships', 'despaired', 'eldes

In [9]:
with open("year_to_all_keywords.json", "w") as outfile:
    json.dump(data, outfile)

## Taking care of temporal datas

In [31]:
with open('year_to_all_keywords.json') as json_file:
    year_to_all_keywords = json.load(json_file)
    
with open("year_to_content.json") as outfile:
    year_to_content = json.load(outfile)

In [32]:
year_to_all_keywords

{'1869': {'chapterwise': {'1gram': ['gandhi', 'gujarati', 'pity'],
   '2gram': ['gandhi grandfather', 'bania caste', 'vow illness'],
   '3gram': ['gandhis belong bania',
    'wife pensioner rajkot',
    'chandrayana2 vow illness']},
  'summarywise': {'1gram': ['gandhi', 'gujarati', 'pity'],
   '2gram': ['gandhi grandfather', 'bania caste', 'vow illness'],
   '3gram': ['gandhis belong bania',
    'wife pensioner rajkot',
    'chandrayana2 vow illness']}},
 '1887': {'chapterwise': {'1gram': ['meat', 'acquaintance', 'weaknesses'],
   '2gram': ['taking meat', 'friend dared', 'explained weak'],
   '3gram': ['insistence abjuring meat',
    'stolen coppers got',
    'friend dared opinion']},
  'summarywise': {'1gram': ['meat', 'acquaintance', 'weaknesses'],
   '2gram': ['taking meat', 'friend dared', 'explained weak'],
   '3gram': ['insistence abjuring meat',
    'stolen coppers got',
    'friend dared opinion']}},
 '1890': {'chapterwise': {'1gram': ['vegetarianism', 'definitions', 'vow'],
  

In [22]:
def get_timeline_interval_content(interval,year_to_content):
    year_to_content = {int(k):v for k,v in year_to_content.items()}
    # initialization
    year_to_content_new = {}
    for k,v in year_to_content.items():
        now = k+interval
        if k in range(k,now):
            year_to_content_new[str(k)+'-'+str(now)] =" "


    for k,v in year_to_content.items():
        now = k+interval
        if k in range(k,now):
            year_to_content_new[str(k)+'-'+str(now)] += year_to_content[k]

    return year_to_content_new

year_to_content_lsts = []
for i in range(1,6):
    year_to_content_lsts.append(get_timeline_interval_content(i,year_to_content))

In [23]:
year_to_content_new = {}
for l in year_to_content_lsts:
    year_to_content_new.update(l)

In [24]:
year_to_content_new.keys()

dict_keys(['1857-1858', '1869-1870', '1887-1888', '1888-1889', '1890-1891', '1891-1892', '1893-1894', '1894-1895', '1896-1897', '1897-1898', '1898-1899', '1900-1901', '1901-1902', '1904-1905', '1906-1907', '1908-1909', '1911-1912', '1912-1913', '1914-1915', '1915-1916', '1917-1918', '1919-1920', '1920-1921', '1921-1922', '1924-1925', '1857-1859', '1869-1871', '1887-1889', '1888-1890', '1890-1892', '1891-1893', '1893-1895', '1894-1896', '1896-1898', '1897-1899', '1898-1900', '1900-1902', '1901-1903', '1904-1906', '1906-1908', '1908-1910', '1911-1913', '1912-1914', '1914-1916', '1915-1917', '1917-1919', '1919-1921', '1920-1922', '1921-1923', '1924-1926', '1857-1860', '1869-1872', '1887-1890', '1888-1891', '1890-1893', '1891-1894', '1893-1896', '1894-1897', '1896-1899', '1897-1900', '1898-1901', '1900-1903', '1901-1904', '1904-1907', '1906-1909', '1908-1911', '1911-1914', '1912-1915', '1914-1917', '1915-1918', '1917-1920', '1919-1922', '1920-1923', '1921-1924', '1924-1927', '1857-1861', '

In [55]:
from summarizer import Summarizer,TransformerSummarizer
year_to_summarizer_new = {}

bert_model = Summarizer()

for key in year_to_content_new.keys():
    year_to_summarizer_new[key] = ''.join(bert_model(year_to_content_new[key], min_length=40))

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [57]:
year_to_summarizer_new.keys()

dict_keys(['1857-1858', '1860-1861', '1869-1870', '1887-1888', '1888-1889', '1890-1891', '1891-1892', '1893-1894', '1894-1895', '1896-1897', '1897-1898', '1900-1901', '1901-1902', '1904-1905', '1906-1907', '1908-1909', '1911-1912', '1912-1913', '1914-1915', '1915-1916', '1917-1918', '1919-1920', '1920-1921', '1921-1922', '1924-1925', '1857-1859', '1860-1862', '1869-1871', '1887-1889', '1888-1890', '1890-1892', '1891-1893', '1893-1895', '1894-1896', '1896-1898', '1897-1899', '1900-1902', '1901-1903', '1904-1906', '1906-1908', '1908-1910', '1911-1913', '1912-1914', '1914-1916', '1915-1917', '1917-1919', '1919-1921', '1920-1922', '1921-1923', '1924-1926', '1857-1860', '1860-1863', '1869-1872', '1887-1890', '1888-1891', '1890-1893', '1891-1894', '1893-1896', '1894-1897', '1896-1899', '1897-1900', '1900-1903', '1901-1904', '1904-1907', '1906-1909', '1908-1911', '1911-1914', '1912-1915', '1914-1917', '1915-1918', '1917-1920', '1919-1922', '1920-1923', '1921-1924', '1924-1927', '1857-1861', '

In [58]:
with open("temporal_year_to_summarizer.json", "w") as outfile:
    json.dump(year_to_summarizer_new, outfile)

In [25]:
with open('temporal_year_to_summarizer.json') as json_file:
    year_to_summarizer_new = json.load(json_file)

In [29]:
year_to_content_new.keys()

dict_keys(['1857-1858', '1869-1870', '1887-1888', '1888-1889', '1890-1891', '1891-1892', '1893-1894', '1894-1895', '1896-1897', '1897-1898', '1898-1899', '1900-1901', '1901-1902', '1904-1905', '1906-1907', '1908-1909', '1911-1912', '1912-1913', '1914-1915', '1915-1916', '1917-1918', '1919-1920', '1920-1921', '1921-1922', '1924-1925', '1857-1859', '1869-1871', '1887-1889', '1888-1890', '1890-1892', '1891-1893', '1893-1895', '1894-1896', '1896-1898', '1897-1899', '1898-1900', '1900-1902', '1901-1903', '1904-1906', '1906-1908', '1908-1910', '1911-1913', '1912-1914', '1914-1916', '1915-1917', '1917-1919', '1919-1921', '1920-1922', '1921-1923', '1924-1926', '1857-1860', '1869-1872', '1887-1890', '1888-1891', '1890-1893', '1891-1894', '1893-1896', '1894-1897', '1896-1899', '1897-1900', '1898-1901', '1900-1903', '1901-1904', '1904-1907', '1906-1909', '1908-1911', '1911-1914', '1912-1915', '1914-1917', '1915-1918', '1917-1920', '1919-1922', '1920-1923', '1921-1924', '1924-1927', '1857-1861', '

In [30]:
year_to_summarizer_new.keys()

dict_keys(['1857-1858', '1860-1861', '1869-1870', '1887-1888', '1888-1889', '1890-1891', '1891-1892', '1893-1894', '1894-1895', '1896-1897', '1897-1898', '1900-1901', '1901-1902', '1904-1905', '1906-1907', '1908-1909', '1911-1912', '1912-1913', '1914-1915', '1915-1916', '1917-1918', '1919-1920', '1920-1921', '1921-1922', '1924-1925', '1857-1859', '1860-1862', '1869-1871', '1887-1889', '1888-1890', '1890-1892', '1891-1893', '1893-1895', '1894-1896', '1896-1898', '1897-1899', '1900-1902', '1901-1903', '1904-1906', '1906-1908', '1908-1910', '1911-1913', '1912-1914', '1914-1916', '1915-1917', '1917-1919', '1919-1921', '1920-1922', '1921-1923', '1924-1926', '1857-1860', '1860-1863', '1869-1872', '1887-1890', '1888-1891', '1890-1893', '1891-1894', '1893-1896', '1894-1897', '1896-1899', '1897-1900', '1900-1903', '1901-1904', '1904-1907', '1906-1909', '1908-1911', '1911-1914', '1912-1915', '1914-1917', '1915-1918', '1917-1920', '1919-1922', '1920-1923', '1921-1924', '1924-1927', '1857-1861', '

In [26]:
year = {
        "chapterwise": {
            "1gram":[],
            "2gram":[],
            "3gram":[]
        },
        "summarywise":{
            "1gram":[],
            "2gram":[],
            "3gram":[]     
        }
}

data2 = {}    
    
for key in year_to_summarizer_new.keys():
    print(key)
    data2[key] = year
    onegram_keywords,twogram_keywords,threegram_keywords = extract_keyword_from_date(year_to_summarizer_new[key])
    onegram_keywords2,twogram_keywords2,threegram_keywords2 = extract_keyword_from_date(year_to_content_new[key])
    #year_to_content[key]
    data2 = copy.deepcopy(data2)
    data2[key]["chapterwise"]['1gram'] = onegram_keywords2
    data2[key]["chapterwise"]['2gram'] = twogram_keywords2
    data2[key]["chapterwise"]['3gram'] = threegram_keywords2
    data2[key]["summarywise"]['1gram'] = onegram_keywords
    data2[key]["summarywise"]['2gram'] = twogram_keywords
    data2[key]["summarywise"]['3gram'] = threegram_keywords

1857-1858
1860-1861


KeyError: '1860-1861'

In [11]:
with open("temporal_year_to_all_keywords.json", "w") as outfile:
    json.dump(data2, outfile)

In [12]:
data2

{'1857-1858': {'chapterwise': {'1gram': ['congress', 'natal', 'pandit'],
   '2gram': ['indian congress', 'organization permanent', 'stay natal'],
   '3gram': ['natal indian congress',
    'bring permanent organization',
    'agitation started 1894']},
  'summarywise': {'1gram': ['mahatmaship', 'meetings', 'indentured'],
   '2gram': ['mahatmaship', 'practice lawyer', 'congress knew'],
   '3gram': ['congress life india',
    'aided agitation started',
    'belaboured master liable']}},
 '1860-1861': {'chapterwise': {'1gram': ['congress', 'natal', 'pandit'],
   '2gram': ['indian congress', 'organization permanent', 'stay natal'],
   '3gram': ['natal indian congress',
    'bring permanent organization',
    'agitation started 1894']},
  'summarywise': {'1gram': ['mahatmaship', 'meetings', 'indentured'],
   '2gram': ['mahatmaship', 'practice lawyer', 'congress knew'],
   '3gram': ['congress life india',
    'aided agitation started',
    'belaboured master liable']}},
 '1869-1870': {'chapte

In [14]:
year_to_all_keywords.update(data2)

In [15]:
year_to_all_keywords.keys()

dict_keys(['1857', '1860', '1869', '1887', '1888', '1890', '1891', '1893', '1894', '1896', '1897', '1900', '1901', '1904', '1906', '1908', '1911', '1912', '1914', '1915', '1917', '1919', '1920', '1921', '1924', '1857-1858', '1860-1861', '1869-1870', '1887-1888', '1888-1889', '1890-1891', '1891-1892', '1893-1894', '1894-1895', '1896-1897', '1897-1898', '1900-1901', '1901-1902', '1904-1905', '1906-1907', '1908-1909', '1911-1912', '1912-1913', '1914-1915', '1915-1916', '1917-1918', '1919-1920', '1920-1921', '1921-1922', '1924-1925', '1857-1859', '1860-1862', '1869-1871', '1887-1889', '1888-1890', '1890-1892', '1891-1893', '1893-1895', '1894-1896', '1896-1898', '1897-1899', '1900-1902', '1901-1903', '1904-1906', '1906-1908', '1908-1910', '1911-1913', '1912-1914', '1914-1916', '1915-1917', '1917-1919', '1919-1921', '1920-1922', '1921-1923', '1924-1926', '1857-1860', '1860-1863', '1869-1872', '1887-1890', '1888-1891', '1890-1893', '1891-1894', '1893-1896', '1894-1897', '1896-1899', '1897-190

In [16]:
with open("final_13_jan.json", "w") as outfile:
    json.dump(year_to_all_keywords, outfile)